In [2]:
import json

In [3]:
# Read the embeddingss (JSON file) into a python dictionary
content = open("/Users/gkhanna/Google Drive/W251DeepLearning/hw6/output_mrpc.jsonl", "r").read()
data = [json.loads(str(item)) for item in content.strip().split('\n')]

In [4]:
# Getting some sense of the dictionary
print("""
    Number of lines in the data list: {}
    Type of each entry: {}
    Keys in each line entry: {}
    Type of the first key (features): {}
    Len of the first key (features): {}
    Type of the first entry in the features list: {}
    Keys in feature list entries: {}
    Type of the first key (token): {}
    Type of the 2nd key (layers): {}
    Number of layers: {}
    Type of each layer: {}
    Keys in each layer: {}
    Type of the first key (index): {}
    Type of the second key (values): {}
    Type of each element in "values": {}
    Number of "values": {}
    """
      .format (len(data), type(data[0]),data[0].keys(), type(data[0]['features']), len(data[0]['features']), 
              type(data[0]['features'][0]),data[0]['features'][0].keys(),type(data[0]['features'][0]['token']),
              type(data[0]['features'][0]['layers']),type(data[0]['features'][0]['layers']), 
               type(data[0]['features'][0]['layers'][0]), data[0]['features'][0]['layers'][0].keys(),
               type(data[0]['features'][0]['layers'][0]['index']),
               type(data[0]['features'][0]['layers'][0]['values']),
               type(data[0]['features'][0]['layers'][0]['values'][0]), 
               len(data[0]['features'][0]['layers'][0]['values'])
                            
              )
     )



    Number of lines in the data list: 3
    Type of each entry: <class 'dict'>
    Keys in each line entry: dict_keys(['linex_index', 'features'])
    Type of the first key (features): <class 'list'>
    Len of the first key (features): 46
    Type of the first entry in the features list: <class 'dict'>
    Keys in feature list entries: dict_keys(['token', 'layers'])
    Type of the first key (token): <class 'str'>
    Type of the 2nd key (layers): <class 'list'>
    Number of layers: <class 'list'>
    Type of each layer: <class 'dict'>
    Keys in each layer: dict_keys(['index', 'values'])
    Type of the first key (index): <class 'int'>
    Type of the second key (values): <class 'list'>
    Type of each element in "values": <class 'float'>
    Number of "values": 768
    


In [5]:
# Printing of a sample line
for item in data[2]['features']:
    print(item['token'], end=" ")

[CLS] are breast pumps covered under my plan ? [SEP] i was wondering where i can find info on breast pumps and how to get one that ' s covered by my insurance . [SEP] 

In [6]:
# Testing of ways to get the location of [SEP]
# We only need the first one to separate out the 2 lines
for index, elem in enumerate(data[0]['features']):
    if (elem['token']=="[SEP]"):
        print(index)


15
45


In [7]:
# Creating lists a and b
# a holds the embeddings for first sentence for the comparison, b holds the second
# We'll use them for the similarity metrics
# The embeddings for a sentence are made by concatenating the embeddings for each word
a = []
b = []
for line_no, linex in enumerate(data):
    a_temp = []
    b_temp = []
    print('Working on line:', line_no)
    # Find the index corresponding to [SEP]
    for index, feature in enumerate(linex['features']):
        if (feature['token'] == '[SEP]'):
            sep_index = index
            print(sep_index)
            break
    for index, feature in enumerate(linex['features']):
        if(feature['token'] == "[CLS]" or feature['token'] == "[SEP]"):
            continue
        if index < sep_index:
            a_temp.extend(feature['layers'][0]['values'])
        else:
            b_temp.extend(feature['layers'][0]['values'])
    a.append(a_temp)
    b.append(b_temp)
    

Working on line: 0
15
Working on line: 1
11
Working on line: 2
9


In [8]:
# Testing if the lists are created fine
len(b[0])


22272

In [9]:
# Changing the lists to np arrays of a fixed size (max of any line)

import numpy as np

length_a = len(sorted(a,key=len, reverse=True)[0])
length_b = len(sorted(b,key=len, reverse=True)[0])

if length_a > length_b:
    length = length_a
else:
    length = length_b
    
a_np=np.array([ai+[0.0]*(length-len(ai)) for ai in a])

b_np=np.array([bi+[0.0]*(length-len(bi)) for bi in b])


In [10]:
# Testing of the arrays are the max len
print(len(a_np[0]))
print(len(b_np[0]))

22272
22272


In [12]:
# Testing one of the arrays
b_np[0]

array([ 0.252011, -0.149837,  0.299606, ..., -0.057238, -0.5152  ,
       -2.014556])

In [13]:
# Testing the cosine similarity metric
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([a_np[2]],[b_np[2]])

array([[0.22518777]])

In [14]:
# Pairwise similarity metric among the arrays
# The diag is the similarity metric between sentences in a and b
cosine_similarity([a_np[0]],[b_np[0]])

array([[0.2702666]])

In [16]:
cosine_similarity(a_np, b_np)

array([[0.2702666 , 0.3519781 , 0.27356778],
       [0.17115618, 0.34138649, 0.23624048],
       [0.15305919, 0.25605092, 0.22518777]])

In [17]:
# Normalizing the embeddings for more precision in the similarity metric
from sklearn import preprocessing
a_np_n = preprocessing.normalize(a_np)
b_np_n = preprocessing.normalize(b_np)

In [18]:
# Cosine similarity between the normalized arrays
cosine_similarity(a_np_n, b_np_n)

array([[0.2702666 , 0.3519781 , 0.27356778],
       [0.17115618, 0.34138649, 0.23624048],
       [0.15305919, 0.25605092, 0.22518777]])